# Functions to generate frame states

In [ ]:
# Although I have made some minor modifications, this part of the code was given to me by Peter Xenopoulos
import lzma
import json

# Function to read .xz archives from ESTA
def read_parsed_demo(filename):
  with lzma.LZMAFile(filename, "rb") as f:
    d = json.load(f)
    return d

def generate_vector_state(frame, map_name):
    """Returns a game state in a dictionary format.

    Args:
        frame (dict) : Dict output of a frame generated from the DemoParser class
        map_name (string): String indicating the map name

    Returns:
        A dict with keys for each feature.
    """
    game_state = {}
    game_state["mapName"] = map_name
    game_state["secondsSincePhaseStart"] = frame["seconds"]
    game_state["bombPlanted"] = frame["bombPlanted"]
    game_state["bombsite"] = frame["bombsite"]
    game_state["totalSmokes"] = len(frame["smokes"])
    game_state["totalFires"] = len(frame["fires"])

    # Team specific info (CT)
    game_state["ctAlive"] = 0
    game_state["ctHp"] = 0
    game_state["ctArmor"] = 0
    game_state["ctHelmet"] = 0
    game_state["ctEq"] = 0
    game_state["ctUtility"] = 0
    game_state["ctEqValStart"] = 0
    game_state["ctBombZone"] = 0
    game_state["defusers"] = 0
    game_state["ctNone"] = 0
    game_state["ctCash"] = 0
    if frame["ct"]["players"] != None:
        for p in frame["ct"]["players"]:
            game_state["ctEqValStart"] += p["equipmentValueFreezetimeEnd"]
            game_state["ctCash"] += p["cash"]
            if p["isAlive"]:
                game_state["ctAlive"] += 1
                game_state["ctHp"] += p["hp"]
                game_state["ctArmor"] += p["armor"]
                game_state["ctHelmet"] += p["hasHelmet"]
                game_state["ctEq"] += p["equipmentValue"]
                game_state["ctUtility"] += p["totalUtility"]
                game_state["defusers"] += p["hasDefuse"]
                if p["isInBombZone"]:
                    game_state["ctBombZone"] += 1
    else: 
        game_state["ctNone"] = 1

    # Team specific info (T)
    game_state["tAlive"] = 0
    game_state["tHp"] = 0
    game_state["tArmor"] = 0
    game_state["tHelmet"] = 0
    game_state["tEq"] = 0
    game_state["tUtility"] = 0
    game_state["tEqValStart"] = 0
    game_state["tHoldingBomb"] = 0
    game_state["tBombZone"] = 0
    game_state["tNone"] = 0
    game_state["tCash"] = 0
    if frame["t"]["players"] != None:
        for p in frame["t"]["players"]:
            game_state["tEqValStart"] += p["equipmentValueFreezetimeEnd"]
            game_state["tCash"] += p["cash"]
            if p["isAlive"]:
                game_state["tAlive"] += 1
                game_state["tHp"] += p["hp"]
                game_state["tArmor"] += p["armor"]
                game_state["tHelmet"] += p["hasHelmet"]
                game_state["tEq"] += p["equipmentValue"]
                game_state["tUtility"] += p["totalUtility"]
                if p["isInBombZone"]:
                    game_state["tBombZone"] += 1
                if p["hasBomb"]:
                    game_state["tHoldingBomb"] = 1
    else: 
        game_state["tNone"] = 1

    return game_state

# Via json files

In [ ]:
# Folder where demos are stored
#path_to_demos = "C:\\Users\\Matias\\Downloads\\esta\\data\\online"
path_to_demos = "..\\input"
path_to_json = "..\\jsons"
# Folder to output dataframe
output = "..\\output"
# Folder with txt file for demos to view
path_to_txt = "..\\Demos_a_revisar.txt"

In [ ]:
'''
Parsing games, compressing json file, compressing it and deleting .dem files
'''
from openpyxl import Workbook
from awpy.parser import DemoParser
import pandas as pd
import os, shutil
import patoolib
import zipfile
from zipfile import ZipFile

os.chdir(path_to_demos)

# Main dataframe
round_state_df = pd.DataFrame()

carpetas = []
for file in os.listdir():
    if ".rar" in file:
        patoolib.extract_archive("%s" % file, outdir="%s" %path_to_demos)

    # Grab demo names
    demos = []
    for demo in os.listdir():
        if (".dem" in demo) and (demo[0]!="x"):
            os.rename(demo, "x" + file.split("vs")[0] + "vs" + demo.split("vs")[1])
            demos += ["x" + file.split("vs")[0] + "vs" + demo.split("vs")[1]]
        if ".json" in demo:
            os.remove(demo)
            
    ### Itero parser over demos
    for demo in demos:
        demo_parser = DemoParser(
        demofile = "%s" % demo,
        parse_rate=128, 
        buy_style="hltv",
        parse_chat = True
        )
    
        try:
            # Parse the demofile, output results to dictionary
            df = demo_parser.parse(return_type="json")
    
            print(f"{demo[1:-4]}")
            os.remove(demo)
            shutil.move(f'x{demo[1:-4]}.json', f"{path_to_json}")
            
        except Exception as e:
            print(e)
            continue
    
    os.remove(file)

In [ ]:
from openpyxl import Workbook
from awpy.parser import DemoParser
import pandas as pd
import os, shutil
import patoolib
import json
import lzma


os.chdir("..\\jsons")

# Main dataframe
round_state_df = pd.DataFrame()

# Grab demo names
demos = []
for demo in os.listdir():
    if ".json" in demo:
        print(f"{demo}")
        with open(demo, encoding="utf-8") as demo_json:
                df = json.load(demo_json)
        #with open(demo, encoding = 'Latin-1') as f:
        #    df = json.load(f)
    # Grab round end frames (last frame of every round)
        frames = []
        iteration = 1
        for ronda in df["gameRounds"]:
            for f in ronda["frames"]:
                if f["clockTime"] == "01:55":
                    frames.append(ronda["frames"][-1])
                    break
        mapa = df["mapName"]
        
    
        # Generate vectors for each frame and pass them to df
        states = []
        for f in frames:
            game_state = generate_vector_state(f, mapa)
            states.append(game_state)
        states = pd.DataFrame(states)
        states["matchID"] = (demo[:-4])
        
        try:
            # Get total freeze time
            freezeTimeEnd = []
            startTick = []
            ctTeam = []
            tTeam = []
            winningSide = []
            for round in df["gameRounds"]:
                freezeTimeEnd.append(round["freezeTimeEndTick"])
                startTick.append(round["startTick"])
                ctTeam.append(round["ctTeam"])
                tTeam.append(round["tTeam"])
                winningSide.append(round["winningSide"])

            states["freezeTimeEndTick"] = freezeTimeEnd
            states["startTick"] = startTick
            states["ctTeam"] = ctTeam
            states["tTeam"] = tTeam
            states["winningSide"] = winningSide
            states["freezeTimeTotal"] = states["freezeTimeEndTick"] - states["startTick"] 
            states["pause"] = 0
            states["player"] = ""
            states["msg"] = ""

            # Iterate over messages, if they include a mention of a pause (either include word "pause" or "tech"), then:
            #    If msgtick >= freezeTimeEndTick ==> pause = 1 in current roundnum + 1

            for msg in df['chatMessages']:

                if msg['params'] is not None:
                    if "tech" in msg["text"] or "pause" in msg["text"]:
                        print(f'There was a pause called by {msg["params"][0]}:  {msg["text"]} at tick {msg["tick"]}' )
                        index = states.index[states["freezeTimeEndTick"]>=msg["tick"]][0]
                        if states["pause"].loc[index] == 0:
                            states.loc[index, "pause"] = 1
                            states.loc[index, "player"] = msg["params"][0]
                            states.loc[index, "msg"] = msg["text"]
                            
                if msg['params'] is None:
                    if "A player disconnected, auto pausing." in msg["text"] or "Waiting for both teams and admin to ready to continue." in msg["text"]:
                        index = states.index[states["freezeTimeEndTick"]>=msg["tick"]][0]
                        if states["pause"].loc[index] == 0:
                            states.loc[index, "pause"] = 1
                            states.loc[index, "player"] = "Admin"
                            states.loc[index, "msg"] = msg["text"]
                        
                            
            for i in range(0,5):
                states[f"ct_p{i+1}"] = ""
                states[f"t_p{i+1}"] = ""   
                states.loc[0:15, f"ct_p{i+1}"] = df["gameRounds"][0]["ctSide"]["players"][i]["playerName"]
                states.loc[0:15, f"t_p{i+1}"]  = df["gameRounds"][0]["tSide"]["players"][i]["playerName"]
                states.loc[15:, f"ct_p{i+1}"]  = df["gameRounds"][0]["tSide"]["players"][i]["playerName"]
                states.loc[15:, f"t_p{i+1}"]  = df["gameRounds"][0]["ctSide"]["players"][i]["playerName"]

        except:
            states["mismatch"] = 0
            
            for msg in df['chatMessages']:

                if msg['params'] is not None:
                    if "tech" in msg["text"] or "pause" in msg["text"]:
                        states.loc[:, "mismatch"] = 1
        
        round_state_df = pd.concat([round_state_df, pd.DataFrame(states)])
        
round_state_df.reset_index(inplace = True)
round_state_df["roundNum"] = round_state_df["index"] + 1
round_state_df.drop("index", axis = 1, inplace = True)

with pd.ExcelWriter(
    f"{output}\\parsed.xlsx",
    mode = "w",
    engine = "xlsxwriter"
) as writer:
    round_state_df.to_excel(writer, index = False, startrow = 1, header = True)